In [1]:
import pandas as pd
from  pandas_profiling import ProfileReport as pr #data visualization
import matplotlib.pyplot as mp
import statsmodels.api as sm
import seaborn as sb
import datetime
import sweetviz as sv #for data visualization
sb.set()

In [2]:
cab_data = pd.read_csv('Cab_Data.csv')
city_list = pd.read_csv('City.csv')
customer_id_data = pd.read_csv('Customer_ID.csv')
transaction_id_data = pd.read_csv('Transaction_ID.csv')

 #Data Conversion for cab_data (from xlsx to datetime)
cab_data['Date of Travel'] = pd.to_datetime(cab_data['Date of Travel'],
                                                  unit = 'D',
                                                  origin = '1899-12-30')

    #Develop Profile Reports for the datasets
# pr(cab_data).to_file('cab_data.html')
# pr(customer_id_data).to_file('customer_id_data.html')
# pr(cab_data).to_file('cab_data.html')
# pr(city_list).to_file('city_list.html')

#     #Develop comparative analysis for the datasets
# sv.compare(cab_data,transaction_id_data).show_html('cab_transaction_comp.html')

   

Summarize dataset:   0%|          | 0/21 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/18 [00:00<?, ?it/s]

...

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

                                             |          | [  0%]   00:00 -> (? left)

Report cab_transaction_comp.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [3]:
# print(cab_data_copy.strftime("%d-%m-%y"))

    #Test - convert Users str to int & increment by 1
#city_list_copy = city_list
# print(city_list_copy.loc[:,'Users'][0])
# print(type(city_list_copy.loc[:,'Users'][0]))
# print(int(city_list_copy.loc[:,'Users'][0].replace(',','')))

# for i in range(len(city_list_copy)):
#     city_list_copy.loc[i].replace(to_replace = city_list_copy.loc[:,'City'][i],
#                                   value = int(city_list_copy.loc[:,'Users'][i].replace(',','')) + 1,
#                                   inplace = True )
#     print(city_list_copy.loc[i])


In [4]:
                    #FAILED TRIALS
# type(cab_data_copy.loc[:,'Date of Travel'][0].item())

# for i in range(len(cab_data_copy)):
#     print(type(cab_data_copy.loc[i,'Date of Travel']))
# print(cab_data_copy.loc[0,'Date of Travel'])

# type(cab_data_copy.loc[4,'Date of Travel'])

# cab_data_copy.loc[4,'Date of Travel']

# for i in range(len(cab_data_copy)):#Iterate through all values in the DataFrame
#     #replace Date of Travel values

# for i in range(len(cab_data_raw_dates)):
#     cab_data_raw_dates[i,'b'] = start_date + datetime.timedelta(cab_data_raw_dates.loc[i,'a'].item() - 42371)
    
cab_data_raw_dates

# def date_calculate(ref):
#     for i in range(len(cab_data_copy)):
#             start_date + datetime.timedelta(cab_data_dates.loc[i,'a'].item() - 42371)

# print(type(cab_data_dates[i]))

# for i in range(len(cab_data_dates)):#Iterate through all values in the DataFrame
# #     cab_data_dates.replace(to_replace = cab_data_dates.loc[i],
# #                          #Compare Date of Travel against Start Date & increment accordingly
# #                          value = start_date + datetime.timedelta(cab_data_dates.loc[i].item() - 42371),
# #                          #Replace value in DataFrame
# #                         inplace = True)
#                         print(type(cab_data_dates[i]))
# cab_data_dates.loc[:,'a']
#     cab_data_copy.replace(to_replace = cab_data_copy.loc[i,'Date of Travel'],
#                          #Compare Date of Travel against Start Date & increment accordingly
#                          value = 'x',
# #                           start_date + datetime.timedelta(cab_data_copy.loc[i,'Date of Travel'].item() - 42371),
#                          #Replace value in DataFrame
#                         inplace = True)

# print(start_date + datetime.timedelta(cab_data.loc[3,'Date of Travel'].item() - 42371)) 

# type(cab_data_copy.loc[5,'Date of Travel'])
# cab_data.replace(to_replace = cab_data.loc[5,"Date of Travel"],
#                        value = start_date + datetime.timedelta(cab_data.loc[3,'Date of Travel'].item() - 42371),
#                        inplace = True)

# cab_data.loc[5]
    

NameError: name 'cab_data_raw_dates' is not defined